In [76]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [77]:
import sys
root = '../../'
sys.path.append(root)
#from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [78]:
X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
X_train_tensor = torch.tensor(X_train)
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)


X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
X_test_tensor = torch.tensor(X_test)
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)


In [79]:
def CreateDataset(root, num_classes, batch_size, train = True):
    if train == True:
        #Create X_train_tensor
        X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
        X_train_tensor = torch.tensor(X_train)

        #Create Y_train_tensor
        y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

        #Create indices
        indices_train = torch.arange(len(X_train_tensor))

        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader

    else:
        X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
        X_test_tensor = torch.tensor(X_test)

        y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)
        y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

        #Create indices
        indices_test = torch.arange(len(X_test_tensor))

        dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return test_loader

    #Missing implementation for Test and Validation
    

In [80]:
train_loader = CreateDataset(root, num_classes = 10, batch_size = 128)
test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = False)


In [81]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, 32),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [82]:
model = CustomNN().to(device)

In [83]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float()
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [84]:
def train_val(device, train_loader, test_loader, train_size, test_size, batch_size, n_classes, bit, num_epoch, lr):


    optimizer = optim.RMSprop(model.parameters(), lr=1e-5, weight_decay = 10 ** -5)

    criterion = DPSHLoss(train_size, n_classes, bit)


    #model.train()

    #Best_mAP = 0

    for epoch in range(num_epoch):

        current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

        print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")

        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = model(image)


            loss = criterion(u, label.float(), ind, lr)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / (train_size / batch_size)

        print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))

    return model
        #if (epoch + 1) % 3 == 0:
            #map = meanAveragePrecision(train_loader, test_loader, model, device)
            #print(map)
            #Best_mAP = validate(config, Best_mAP, test_loader, dataset_loader, net, bit, epoch, num_dataset)


In [85]:
trained_model = train_val(device, train_loader, test_loader, train_size = 45000, test_size = 10000, batch_size = 128, n_classes = 10, bit = 32, num_epoch = 150, lr = 0.1)

DPSH[ 1/150][10:19:17] bit:32, dataset:CIFAR, training... loss:0.74281
DPSH[ 2/150][10:19:19] bit:32, dataset:CIFAR, training... loss:0.70507
DPSH[ 3/150][10:19:20] bit:32, dataset:CIFAR, training... loss:0.66893
DPSH[ 4/150][10:19:22] bit:32, dataset:CIFAR, training... loss:0.63625
DPSH[ 5/150][10:19:25] bit:32, dataset:CIFAR, training... loss:0.60787
DPSH[ 6/150][10:19:27] bit:32, dataset:CIFAR, training... loss:0.58551
DPSH[ 7/150][10:19:29] bit:32, dataset:CIFAR, training... loss:0.56877
DPSH[ 8/150][10:19:31] bit:32, dataset:CIFAR, training... loss:0.55604
DPSH[ 9/150][10:19:32] bit:32, dataset:CIFAR, training... loss:0.54633
DPSH[10/150][10:19:34] bit:32, dataset:CIFAR, training... loss:0.53826
DPSH[11/150][10:19:37] bit:32, dataset:CIFAR, training... loss:0.53153
DPSH[12/150][10:19:40] bit:32, dataset:CIFAR, training... loss:0.52562
DPSH[13/150][10:19:43] bit:32, dataset:CIFAR, training... loss:0.52045
DPSH[14/150][10:19:45] bit:32, dataset:CIFAR, training... loss:0.51568
DPSH[1

In [86]:
trained_model = trained_model.cpu()

In [87]:
hash_train = (trained_model(X_train_tensor)).sign()
hash_train = hash_train.detach().numpy()
hash_test = (trained_model(X_test_tensor)).sign()
hash_test = hash_test.detach().numpy()
y_train
y_test

array([3, 8, 8, ..., 5, 1, 7], dtype=int64)

In [88]:
map = meanAveragePrecision(training_hashes = hash_train, test_hashes = hash_test, test_labels = y_test, training_labels = y_train)

100%|██████████| 10000/10000 [01:25<00:00, 116.32it/s]


In [89]:
map

0.854891885459112